In [ ]:
import pysrt
import os
from deep_translator import GoogleTranslator
import speech_recognition as sr
import moviepy.editor as mp
import datetime
import pandas as pd

#INPUT
FilePath = "" # Path of the file or the file name if present in the same directory
# Example : "Desktop\Folder1\video.mp4"
# If Error occurs try adding the path like this r"Desktop\Folder1\video.mp4" (With a r at the start of the string)

TargetLanguage = 'ur' # The language in which you want subtitles to be for example en = english ur = urdu and so on
# For more langauge put their ISO code in qoutes  NOTE: Supported Languages along with their supported qoutes are provided in README

base = os.path.splitext(FilePath)[0]
clip  = mp.VideoFileClip(FilePath)
clip.audio.write_audiofile(base + '.wav') 

r = sr.Recognizer()
audio = sr.AudioFile(base + '.wav')

from moviepy.editor import VideoFileClip
clip = VideoFileClip(FilePath)
length = clip.duration

file = pysrt.SubRipFile()
SUBTITILES = ''

start_time = datetime.datetime(100,1,1,0,0,0)
starttime = 0
endtime = 0
Duration = 7
block = 0
Subs = []

while (starttime <= length):
    
    if (starttime + 7 <= length):
        with audio as source:
            audio_file = r.record(source,duration = Duration,offset = starttime)
    
        try:
            result = r.recognize_google(audio_file)
            translated = GoogleTranslator(source='auto', target= TargetLanguage).translate(result)
            Subs.append(translated)
        except:
            translated = ''
            result = ''
        
        st = str(start_time.time())
        start_time = start_time + datetime.timedelta(0, Duration)
        starttime = starttime + Duration
        et = str(start_time.time())

        SUBTITILES += str(block)
        SUBTITILES += '\n'
        SUBTITILES += st
        SUBTITILES += ' --> '
        SUBTITILES += et
        SUBTITILES += '\n'
        SUBTITILES += translated
        SUBTITILES += '.'
        SUBTITILES += '\n'
        SUBTITILES += '\n'

        block = block + 1
    
        if (starttime + 7 > length):
            RemainingTime = int(length - starttime)
            with audio as source:
                audio_file = r.record(source,duration = RemainingTime,offset = starttime)
    
            try:
                result = r.recognize_google(audio_file)
                translated = GoogleTranslator(source='auto', target= TargetLanguage).translate(result)
                Subs.append(translated)
            except:
                translated = ''
                result = ''
            st = str(start_time.time())
            start_time = start_time + datetime.timedelta(0, RemainingTime)
            starttime = starttime + RemainingTime
            et = str(start_time.time())
            
            SUBTITILES += str(block)
            SUBTITILES += '\n'
            SUBTITILES += st
            SUBTITILES += ' --> '
            SUBTITILES += et
            SUBTITILES += '\n'
            SUBTITILES += translated
            SUBTITILES += '.'
            SUBTITILES += '\n'
            SUBTITILES += '\n'
            
            block = block + 1
            break
    else:
        RemainingTime = int(length - starttime)
        with audio as source:
            audio_file = r.record(source,duration = RemainingTime,offset = starttime)
    
        try:
            result = r.recognize_google(audio_file)
            translated = GoogleTranslator(source='auto', target= TargetLanguage).translate(result)
            Subs.append(translated)
        except:
            translated = ''
            result = ''
        st = str(start_time.time())
        start_time = start_time + datetime.timedelta(0, RemainingTime)
        starttime = starttime + RemainingTime
        et = str(start_time.time())
        
        SUBTITILES += str(block)
        SUBTITILES += '\n'
        SUBTITILES += st
        SUBTITILES += ' --> '
        SUBTITILES += et
        SUBTITILES += '\n'
        SUBTITILES += translated
        SUBTITILES += '.'
        SUBTITILES += '\n'
        SUBTITILES += '\n'
      
        block = block + 1
        break

with open(base + '.txt', "w", encoding="utf-8") as f:
    f.write(SUBTITILES)
os.remove(base + ".wav")
os.rename(base + '.txt', base + '.srt')